In [7]:
import os
import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
from rasterio.plot import show
from rasterio.features import geometry_mask
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Function to normalize bands
def normalize_band(band):
    return band / np.max(band)

# Function to calculate NDVI
def calculate_ndvi(red_band, nir_band):
    ndvi = (nir_band - red_band) / (nir_band + red_band)
    return ndvi

# Paths to Sentinel-1 and Sentinel-2 directories (replace with your paths)
sentinel1_dir = r"C:\Users\rishi\OneDrive\Desktop\sentinel_1"
sentinel2_dir = r"C:\Users\rishi\OneDrive\Desktop\sentinel_2"
shapefile_path = r"C:\Users\rishi\OneDrive\Desktop\crop_data_shapefile\merged_crop_data.shp"

# List all Sentinel-1 and Sentinel-2 files
sentinel1_files = [os.path.join(sentinel1_dir, f) for f in os.listdir(sentinel1_dir) if f.endswith('.tif')]
sentinel2_files = [os.path.join(sentinel2_dir, f) for f in os.listdir(sentinel2_dir) if f.endswith('.tif')]

# Load the shapefile using GeoPandas
gdf = gpd.read_file(shapefile_path)

# Extract features from each polygon in the shapefile
patch_size = 64  # Increased the size of patches to extract
features = []
labels = []

for sentinel1_path, sentinel2_path in zip(sentinel1_files, sentinel2_files):
    # Open and read Sentinel-1 data (assuming multi-band)
    with rasterio.open(sentinel1_path) as src:
        sentinel1_data = src.read()  # Read all bands
        sentinel1_meta = src.meta

    # Open and read Sentinel-2 data (all bands)
    with rasterio.open(sentinel2_path) as src:
        sentinel2_data = src.read()  # Read all bands
        sentinel2_meta = src.meta

    # Ensure the coordinate reference systems match
    gdf = gdf.to_crs(sentinel1_meta['crs'])

    # Extract individual bands from Sentinel-2 data
    sentinel2_red = sentinel2_data[3].astype(float)  # Red band (Band 4)
    sentinel2_nir = sentinel2_data[7].astype(float)  # Near Infrared band (Band 8)

    # Normalize Sentinel-2 bands
    sentinel2_red_norm = normalize_band(sentinel2_red)
    sentinel2_nir_norm = normalize_band(sentinel2_nir)

    # Calculate NDVI using Sentinel-2 bands
    ndvi = calculate_ndvi(sentinel2_red_norm, sentinel2_nir_norm)

    for idx, row in gdf.iterrows():
        geom = row['geometry']
        label = row['layer']  # Replace with the actual column name for crop types

        # Create a mask for the polygon
        mask = geometry_mask([geom], transform=sentinel1_meta['transform'], invert=True, out_shape=(sentinel1_meta['height'], sentinel1_meta['width']))

        # Extract patches of Sentinel-1 and Sentinel-2 data
        for i in range(0, sentinel1_meta['height'], patch_size):
            for j in range(0, sentinel1_meta['width'], patch_size):
                if mask[i:i+patch_size, j:j+patch_size].sum() > 0:  # Ensure there is some data in the patch
                    sentinel1_patch = sentinel1_data[:, i:i+patch_size, j:j+patch_size]
                    sentinel2_patch = np.array([sentinel2_red[i:i+patch_size, j:j+patch_size], sentinel2_nir[i:i+patch_size, j:j+patch_size]])
                    ndvi_patch = ndvi[i:i+patch_size, j:j+patch_size]

                    if sentinel1_patch.shape[1] == patch_size and sentinel1_patch.shape[2] == patch_size:
                        combined_patch = np.concatenate((sentinel1_patch, sentinel2_patch, np.expand_dims(ndvi_patch, axis=0)), axis=0)
                        features.append(combined_patch)
                        labels.append(label)

                # Save the fused patch
                        fused_patch_path = os.path.join(output_dir, f'fused_patch_{idx}_{i}_{j}.tif')
                        with rasterio.open(
                            fused_patch_path, 'w',
                            driver='GTiff',
                            height=combined_patch.shape[1],
                            width=combined_patch.shape[2],
                            count=combined_patch.shape[0],
                            dtype=combined_patch.dtype,
                            crs=sentinel1_meta['crs'],
                            transform=sentinel1_meta['transform']
                        ) as dst:
                            for band in range(combined_patch.shape[0]):
                                dst.write(combined_patch[band, :, :], band + 1)

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Encode labels to integers
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Standardize features
scaler = StandardScaler()
for i in range(features.shape[1]):
    features[:, i, :, :] = scaler.fit_transform(features[:, i, :, :].reshape(-1, features[:, i, :, :].shape[-1])).reshape(features[:, i, :, :].shape)

# Handle class imbalance with SMOTE
original_shape = features.shape
features_flat = features.reshape(features.shape[0], -1)
smote = SMOTE(random_state=42)
features_resampled, labels_resampled = smote.fit_resample(features_flat, labels_encoded)
features_resampled = features_resampled.reshape(-1, original_shape[1], original_shape[2], original_shape[3])

# Convert labels to categorical
labels_resampled = to_categorical(labels_resampled)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_resampled, labels_resampled, test_size=0.2, random_state=42)

# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(features_resampled.shape[1], features_resampled.shape[2], features_resampled.shape[3])))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(labels_resampled.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=25, validation_data=(X_test, y_test))

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Print classification report
print(classification_report(y_true_classes, y_pred_classes, target_names=label_encoder.classes_))


C:\Users\rishi\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 14s 121ms/step - accuracy: 0.3132 - loss: 1.6203 - val_accuracy: 0.5808 - val_loss: 1.1207
Epoch 2/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.4973 - loss: 1.1850 - val_accuracy: 0.5329 - val_loss: 1.0686
Epoch 3/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.5179 - loss: 1.0815 - val_accuracy: 0.5389 - val_loss: 1.0417
Epoch 4/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.5257 - loss: 1.0392 - val_accuracy: 0.6108 - val_loss: 1.0055
Epoch 5/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.5636 - loss: 1.0047 - val_accuracy: 0.5090 - val_loss: 1.0613
Epoch 6/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.4979 - loss: 1.0441 - val_accuracy: 0.5629 - val_loss: 1.0066
Epoch 7/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.5419 - loss: 1.0092 - val_accuracy: 0.5629 - val_loss: 1.0032
Epoch 8/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.5845 - loss: 0.9599 - val_accuracy: 0.5329 -

In [2]:
import os
import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
from rasterio.plot import show
from rasterio.features import geometry_mask
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Function to normalize bands
def normalize_band(band):
    return band / np.max(band)

# Function to calculate NDVI
def calculate_ndvi(red_band, nir_band):
    ndvi = (nir_band - red_band) / (nir_band + red_band)
    return ndvi


sentinel1_dir = r"C:\Users\rishi\OneDrive\Desktop\sentinel_1"
sentinel2_dir = r"C:\Users\rishi\OneDrive\Desktop\sentinel_2"
shapefile_path = r"C:\Users\rishi\OneDrive\Desktop\crop_data_shapefile\merged_crop_data.shp"
output_dir = r"C:\Users\rishi\OneDrive\Desktop\fused data"  

# Create the output directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

# List all Sentinel-1 and Sentinel-2 files
sentinel1_files = [os.path.join(sentinel1_dir, f) for f in os.listdir(sentinel1_dir) if f.endswith('.tif')]
sentinel2_files = [os.path.join(sentinel2_dir, f) for f in os.listdir(sentinel2_dir) if f.endswith('.tif')]

# Load the shapefile using GeoPandas
gdf = gpd.read_file(shapefile_path)

# Extract features from each polygon in the shapefile
patch_size = 64  # Increased the size of patches to extract
features = []
labels = []

for sentinel1_path, sentinel2_path in zip(sentinel1_files, sentinel2_files):
    # Open and read Sentinel-1 data (assuming multi-band)
    with rasterio.open(sentinel1_path) as src:
        sentinel1_data = src.read()  # Read all bands
        sentinel1_meta = src.meta

    # Open and read Sentinel-2 data (all bands)
    with rasterio.open(sentinel2_path) as src:
        sentinel2_data = src.read()  # Read all bands
        sentinel2_meta = src.meta

    # Ensure the coordinate reference systems match
    gdf = gdf.to_crs(sentinel1_meta['crs'])

    # Extract individual bands from Sentinel-2 data
    sentinel2_red = sentinel2_data[3].astype(float)  # Red band (Band 4)
    sentinel2_nir = sentinel2_data[7].astype(float)  # Near Infrared band (Band 8)

    # Normalize Sentinel-2 bands
    sentinel2_red_norm = normalize_band(sentinel2_red)
    sentinel2_nir_norm = normalize_band(sentinel2_nir)

    # Calculate NDVI using Sentinel-2 bands
    ndvi = calculate_ndvi(sentinel2_red_norm, sentinel2_nir_norm)

    for idx, row in gdf.iterrows():
        geom = row['geometry']
        label = row['layer']  # Replace with the actual column name for crop types

        # Create a mask for the polygon
        mask = geometry_mask([geom], transform=sentinel1_meta['transform'], invert=True, out_shape=(sentinel1_meta['height'], sentinel1_meta['width']))

        # Extract patches of Sentinel-1 and Sentinel-2 data
        for i in range(0, sentinel1_meta['height'], patch_size):
            for j in range(0, sentinel1_meta['width'], patch_size):
                if mask[i:i+patch_size, j:j+patch_size].sum() > 0:  # Ensure there is some data in the patch
                    sentinel1_patch = sentinel1_data[:, i:i+patch_size, j:j+patch_size]
                    sentinel2_patch = np.array([sentinel2_red[i:i+patch_size, j:j+patch_size], sentinel2_nir[i:i+patch_size, j:j+patch_size]])
                    ndvi_patch = ndvi[i:i+patch_size, j:j+patch_size]

                    if sentinel1_patch.shape[1] == patch_size and sentinel1_patch.shape[2] == patch_size:
                        combined_patch = np.concatenate((sentinel1_patch, sentinel2_patch, np.expand_dims(ndvi_patch, axis=0)), axis=0)
                        features.append(combined_patch)
                        labels.append(label)

                        # Save the fused patch
                        fused_patch_path = os.path.join(output_dir, f'fused_patch_{idx}_{i}_{j}.tif')
                        with rasterio.open(
                            fused_patch_path, 'w',
                            driver='GTiff',
                            height=combined_patch.shape[1],
                            width=combined_patch.shape[2],
                            count=combined_patch.shape[0],
                            dtype=combined_patch.dtype,
                            crs=sentinel1_meta['crs'],
                            transform=sentinel1_meta['transform']
                        ) as dst:
                            for band in range(combined_patch.shape[0]):
                                dst.write(combined_patch[band, :, :], band + 1)

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Encode labels to integers
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Standardize features
scaler = StandardScaler()
for i in range(features.shape[1]):
    features[:, i, :, :] = scaler.fit_transform(features[:, i, :, :].reshape(-1, features[:, i, :, :].shape[-1])).reshape(features[:, i, :, :].shape)

# Handle class imbalance with SMOTE
original_shape = features.shape
features_flat = features.reshape(features.shape[0], -1)
smote = SMOTE(random_state=42)
features_resampled, labels_resampled = smote.fit_resample(features_flat, labels_encoded)
features_resampled = features_resampled.reshape(-1, original_shape[1], original_shape[2], original_shape[3])

# Convert labels to categorical
labels_resampled = to_categorical(labels_resampled)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_resampled, labels_resampled, test_size=0.2, random_state=42)

# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(features_resampled.shape[1], features_resampled.shape[2], features_resampled.shape[3])))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(labels_resampled.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=25, validation_data=(X_test, y_test))

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Print classification report
print(classification_report(y_true_classes, y_pred_classes, target_names=label_encoder.classes_))


C:\Users\rishi\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 8s 58ms/step - accuracy: 0.3291 - loss: 1.5857 - val_accuracy: 0.5449 - val_loss: 1.1128
Epoch 2/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5006 - loss: 1.1360 - val_accuracy: 0.5210 - val_loss: 1.0730
Epoch 3/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.5225 - loss: 1.0778 - val_accuracy: 0.5150 - val_loss: 1.0523
Epoch 4/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5528 - loss: 1.0590 - val_accuracy: 0.5329 - val_loss: 1.0671
Epoch 5/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.5134 - loss: 1.0446 - val_accuracy: 0.5389 - val_loss: 1.0264
Epoch 6/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5199 - loss: 1.0503 - val_accuracy: 0.5928 - val_loss: 1.0349
Epoch 7/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5863 - loss: 1.0032 - val_accuracy: 0.5090 - val_loss: 1.0588
Epoch 8/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5867 - loss: 0.9840 - val_accuracy: 0.5389 - v